In [57]:
# Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import roc_curve, precision_recall_curve
import matplotlib.pyplot as plt

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
loewe = pd.read_csv("/content/drive/MyDrive/CellLine/oneilaverageloewe.csv")

In [60]:
# loewe = pd.read_csv("/content/drive/MyDrive/depmap cells/loewe.csv")
# loewe = pd.read_csv("/content/drive/MyDrive/depmap cells/deepsynloewe.csv")
rna = pd.read_csv("/content/drive/MyDrive/depmap cells/rna.csv")
mut = pd.read_csv("/content/drive/MyDrive/depmap cells/MUTFINAL.csv")
cnv = pd.read_csv("/content/drive/MyDrive/depmap cells/cnv.csv")
dep= pd.read_csv("/content/drive/MyDrive/depmap cells/dependency.csv")


In [61]:
mut = pd.read_csv("/content/drive/MyDrive/depmap cells/MUTFINAL.csv")

In [62]:
#for bliss-loewe-...
CELL=["KPL1","ZR751","COLO320DM","DLD1","LNCAP", "A375", "A427", "ES2", "HCT116", "MDAMB436", "OV90", "RKO", "RPMI7951", "T47D", 'UWB1289', 'VCAP', 'CAOV3', 'NCIH460',"A2058","A2780","HT144","HT29","LOVO","MSTO","NCIH1650","NCIH2122","NCIH23","NCIH520","OVCAR3","PA1","SKMEL30","SKMES1","SKOV3","SW620",
"SW837","UACC62"]

In [63]:
#for rna-cnv-...
cell=["KPL1","ZR751","COLO320","DLD1","LNCAPCLONEFGC", "A375", "A427", "ES2", "HCT116", "MDAMB436", "OV90", "RKO", "RPMI7951", "T47D", 'UWB1289', 'VCAP', 'CAOV3', 'NCIH460',"A2058","A2780","HT144","HT29","LOVO","MSTO211H","NCIH1650","NCIH2122","NCIH23","NCIH520","OVCAR3","PA1","SKMEL30","SKMES1","SKOV3","SW620",
"SW837","UACC62"]

In [64]:
cellsp=sorted(CELL)
cellsr=sorted(cell)

In [65]:
loewe=loewe[cellsp]
rna=rna[cellsr]
cnv=cnv[cellsr]
mut=mut[cellsr]
dep=dep[cellsr]

In [66]:
loewe.columns = range(loewe.shape[1])

In [67]:
global q,t
q=[0 for i in range(7)]
t=[0 for i in range(7)]
def NO(mat):
  mat = pd.DataFrame(mat)
  LEN=len(mat.iloc[:,0])
  N =pd.DataFrame(0.0, index=range(LEN), columns=range(LEN))
  K = np.empty((LEN,LEN), dtype=float, order='C')
  for i in range(LEN):
    for j in range(LEN):
      N[i][j] = np.linalg.norm(mat.iloc[i,:]-mat.iloc[j,:])
  for i in range(LEN):
    for j in range(LEN):
      if N.iloc[i,j]>600:
        N.iloc[i,j]=600
  for i in range(LEN):
    for j in range(LEN):
      N.iloc[i,j] = 600-N.iloc[i,j]
  N=np.array(N)
  return(N)

In [68]:
loewenorm =NO(loewe.T)

In [69]:
def sim_mut(x,y):
  mutcor =  np.empty((np.shape(x)[0], np.shape(y)[0]))
  for i in range(np.shape(x)[0]):
    for j in range(np.shape(y)[0]):
      m11 = m10= m01=0
      for k in range(len(x[0,:])):
          #print(len(x[0,:]))
          if y[j,k]==1 and x[i,k]==1:
            m11=m11+1
          elif y[j,k]==0 and x[i,k]==1:
            m10=m10+1
          elif x[i,k]==0 and y[j,k]==1:
            m01=m01+1
      if(m11+m10+m01)!=0:
        mutcor[i][j]=float(float(m11)/float(m11+m10+m01))
      else:
        #print(i,j,k)
        mutcor[i][j] = 0
  return mutcor

In [70]:
mut = mut.to_numpy().T

In [71]:
mutcorr=sim_mut(mut,mut)

In [72]:
rnacorr=rna.corr()
cnvcorr=cnv.corr()
depcorr=dep.corr()

In [73]:
rna = rna.to_numpy().T
cnv = cnv.to_numpy().T
dep = dep.to_numpy().T

In [74]:
import scipy.stats
import numpy as np

def naive_pearson_cor(X, Y):
    result = np.zeros(shape=(X.shape[1], Y.shape[1]))
    for i in range(X.shape[1]):
      for j in range(Y.shape[1]):
          r, _ = scipy.stats.pearsonr(X[:,i], Y[:,j])
          result[i,j] = r
    return result

In [75]:
from sklearn.metrics.pairwise import paired_manhattan_distances
import copy
from sklearn.metrics import precision_score
from sklearn.metrics import average_precision_score, precision_recall_curve
# from sklearn.metrics import auc, plot_precision_recall_curve
import matplotlib.pyplot as plt
def Threshold(Pred , cri):
  #print(len(cri))
  tp = 0
  fp = 0
  tn = 0
  fn = 0
  c1=0
  c0=0
  c=0
  P=[]
  C = []
  for i in range(len(Pred)):
    #i = i*3
    if Pred[i] > 0:
      P.append(1)
    else:
      P.append(0)
  for j in cri:
    if j > 0:
      C.append(1)
    else:
      C.append(0)
  for k in range(len(P)):
    if P[k] == 1 and C[k] == 1:
      tp +=1
    if P[k] == 1 and C[k] != 1:
      fp +=1
    if P[k] !=1 and C[k] != 1:
      tn +=1
    if P[k] != 1 and C[k] == 1:
      fn +=1
  TP.append(tp)
  FP.append(fp)
  TN.append(tn)
  FN.append(fn)
  v = tp/(tp+fp)
  d = tp/(tp+fn)
  pre.append(tp/(tp+fp))
  R.append(tp/(tp+fn))
  kappa = (2*((tp*tn)-(fn*fp)))/(((tp+fp)*(fp+tn))+((tp+fn)*(fn+tn)))
  KAP.append(kappa)
  F.append(2*((v*d)/(v+d)))

In [76]:
from scipy import stats

In [77]:
def nearest(k,x,index,cri,trvia):
# nearest(pred.T,X_train,test_index,X,train_index)
  cri = pd.DataFrame(cri)
  MSEM=0.0
  global cells
  global s
  global cls
  for i in range(len(k)):
    m1=0
    mse=0
    l=0
    r=0
    G=[]
    pos=0
    neg=0
    trind=[]
    trsyn=[]

    s=[]
    cls=[0] * len(x[0])
    s=[]
    for l in range(len(x[0])):
      ind = np.argpartition(k[i], -18)[-18:]
      NEI=18
      top = k[i][ind]
      trind= trvia[ind]
      pos=0
      neg=0
      m=0
      m1=0
      trsyn=[]
      sort=pd.DataFrame()
      After =pd.DataFrame()
      for ppp in range(len(trind)):
        trsyn.append(cri.iloc[trind[ppp],l])
      for pp in range(len(trind)):
        sort.loc[0,pp]= ind[pp]
        sort.loc[1,pp]=trind[pp]
        sort.loc[2,pp]=trsyn[pp]
      sort=sort.sort_values(by=2, ascending=False, axis=1)
      sort = pd.DataFrame(sort.values, columns = range(18) )
      co=0
      for qq in range(int(NEI/4), int(NEI*(3/4)+1)):
        After.loc[0,co]=sort.loc[0,qq]
        After.loc[1,co]=sort.loc[2,qq]
        co+=1
      ind=After.iloc[0,:]
      ind = np.asarray(ind, dtype = 'int')
      top = k[i][ind]
      for h in range(len(top)):
        m1 += top[h]

      m=0
      for j in range(len(ind)):
        m += top[j]*x[ind[j],l]
        y=m/m1
      s.append(y)
    mse =mean_squared_error(s,cri.iloc[index[i],:])

    MSEM += mse
    cells[index[i]] = s
    P.append(mse)
    coef,value = scipy.stats.pearsonr(s,cri.iloc[index[i],:])
    COEF.append(coef)
    VALUE.append(value)
    Threshold(s,cri.iloc[index[i],:])
    sum=0

    for lll in range(len(s)):
      sum += abs(cri.iloc[index[i],lll] - s[lll])
    error = sum/len(s)
    MAE.append(error)
    est,value = stats.pearsonr(s,cri.iloc[index[i],:])
    pearsonr.append(est)
    est1,value=stats.spearmanr(s,cri.iloc[index[i],:])
    spearmanr.append(est1)

In [78]:
def store_np(data, filename):
  cols = np.arange(data.shape[1])
  index = np.arange(data.shape[0])
  df = pd.DataFrame(np.array(data), index=index, columns=cols)
  df.to_csv(filename)

In [79]:
def SYNcombination(T1,T2,NORM):
  L=len(T1)*len(T2)
  com= np.zeros(L)
  CO=0
  for i in range(len(T1)):
    for j in range(len(T2)):
      com[CO]= NORM[i,j]
      CO=CO+1

  return com

In [80]:
def traincombination(R,M,C,D):
  l = len(R[0,:])
  ll=len(R[0,:])

  p = (l*ll)
  colcounter = 0
  Xtrain = np.zeros([int(l*8),int(p)])
  for i in range(l):
    for j in range(l):
      x = np.append(R[:,i],(M[:,i],C[:,i],D[:,i],R[:,j],M[:,j],C[:,j],D[:,j]))
      Xtrain[:,colcounter] = x
      colcounter=colcounter+1

  return Xtrain

In [81]:
def testcombination(rtr,mtr,ctr,dtr,rts,mts,cts,dts):
  l = len(rtr[0,:])

  ll=len(rts[0,:])

  p = (l*ll)
  colcounter = 0
  Xtrain = np.zeros([int(l*8),int(p)])
  for i in range(l):
    for j in range(ll):
      x = np.append(rtr[:,i],(mtr[:,i],ctr[:,i],dtr[:,i],rts[:,j],mts[:,j],cts[:,j],dts[:,j]))
      Xtrain[:,colcounter] = x
      colcounter=colcounter+1
  return Xtrain


In [82]:
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from keras.callbacks import ModelCheckpoint
def NNN(Xtrain,Xtest,Ytrain,Ytest):

  model = Sequential()
  model.add(Input(shape=(216,)))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  #model.add(Dropout(.25))
  model.add(Dense(32, activation='relu'))
  #model.add(Dropout(.25))
  model.add(Dense(16, activation='relu'))
  model.add(Dense(1))

  model.compile('adam', 'mse')
  model.fit(Xtrain, Ytrain, epochs=200)
  model.predict(Xtest)
  return(model.predict(Xtest))

In [83]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import scipy.stats
import numpy as np

#/content/drive/MyDrive/Kfold/

def CrossVal(X,splits=6):
  global mut
  global cnv
  global rna
  global dep
  global eff
  X=np.array(X.T)
  kf = KFold(n_splits=splits,shuffle=False)
  for train_index, test_index in kf.split(mut):
    # train_index = np.array([0,1,2,3,4,5,7,8,9,10,11,12,13,15,16,18,19,20,21,22,28,29,30,32,33,34,35])
    # test_index = np.array([23,24,25,27,26,31,6,17])

    rna_train, rna_test = rna[train_index],rna[test_index]
    rna_trainf = pd.DataFrame(rna_train)
    rna_train_corr = rna_trainf.T.corr()
    rna_train_corr=np.array(rna_train_corr).reshape(-1)
    # print(rna_train_corr.shape)
    cnv_train, cnv_test = cnv[train_index],cnv[test_index]
    cnv_trainf = pd.DataFrame(cnv_train)
    cnv_train_corr = cnv_trainf.T.corr()
    cnv_train_corr=np.array(cnv_train_corr).reshape(-1)

    dep_train, dep_test = dep[train_index],dep[test_index]
    dep_trainf = pd.DataFrame(dep_train)
    dep_train_corr = dep_trainf.T.corr()
    dep_train_corr=np.array(dep_train_corr).reshape(-1)

    mut_train, mut_test = mut[train_index],mut[test_index]
    mut_train_corr =sim_mut(mut_train,mut_train).reshape(-1)

    X_train, X_test = X[train_index],X[test_index]
    X_train_corr = NO(X_train).reshape(-1)
    # input_train= traincombination(rna_train_corr,mut_train_corr,cnv_train_corr,dep_train_corr)
    input_train = np.vstack((mut_train_corr,dep_train_corr)).T
    # input_train = np.vstack((mut_train_corr)).T
    modelb = LinearRegression().fit(input_train,X_train_corr)
    # print("coeff:", modell.coef_, modell.intercept_)
    rnacorr = naive_pearson_cor(rna_train.T,rna_test.T).reshape(-1)
    rnacorr = np.nan_to_num(rnacorr)

    cnvcorr = naive_pearson_cor(cnv_train.T,cnv_test.T).reshape(-1)
    depcorr = naive_pearson_cor(dep_train.T,dep_test.T).reshape(-1)
    depcorr = np.nan_to_num(depcorr)

    mutcorr = sim_mut(mut_train,mut_test).reshape(-1)
    # input_test = np.vstack((rnacorr,mutcorr,cnvcorr,depcorr)).T
    input_test = np.vstack((mutcorr,depcorr)).T
    pred1 = modelb.predict(input_test)
    pred = np. reshape(pred1, (len(train_index), len(test_index)))
    nearest(pred.T,X_train,test_index,X,train_index)



  return

In [84]:
from numpy.ma.core import count
from statistics import mean
import time
global Z
global P
AUC=[]
AUPR=[]
MAE=[]
pearsonr=[]
spearmanr=[]
P=[]
orgP=[]
COEF=[]
VALUE=[]
global PER
PER=[]
global pre
global KAP
global R
global F
global TP
global FP
global TN
global FN
cells = pd.DataFrame()
CLS=pd.DataFrame()
#=np.array([,])
for Q in range(1):
  pre=[]
  R=[]
  KAP=[]
  #cells=[]
  F=[]
  TP =[]
  TN =[]
  FP =[]
  FN =[]
  FFSC=[]
  FMSE=[]
  FPRE=[]
  FREC=[]
  FPEASON=[]
  FSPE=[]
  CrossVal(loewe)
  FMSE.append(mean(P))
  FFSC.append(mean(F))
  FREC.append(mean(R))
  FPRE.append(mean(pre))
  FPEASON.append(mean(pearsonr))
  FSPE.append(mean(spearmanr))
print('precision:',mean(pre))
print('recall:',mean(FREC))
print('f score',mean(FFSC))
print('MSE:',mean(FMSE))
print('PEARSON:',mean(FPEASON))
print('SPEARMAN:',mean(FSPE))
print('TP',mean(TP))
print('TN',mean(TN))
print('FP',mean(FP))
print('FN',mean(FN))

precision: 0.7034707904396339
recall: 0.6523567679117237
f score 0.6719575707288438
MSE: 149.27918156530683
PEARSON: 0.6442931517252847
SPEARMAN: 0.6413229483565528
TP 156.88888888888889
TN 273.6666666666667
FP 66.30555555555556
FN 86.13888888888889
